In [1]:

import os
import numpy as np
import glob
import cv2
from sklearn.utils import shuffle
from tqdm import tqdm
import tensorflow as tf
from makiflow.augmentation import AffineAugment, ElasticAugment, ImageCutter, Data, FlipAugment


In [4]:
def get_masks_and_images(path):
    masks = glob.glob(path + '/masks/*.bmp')
    images = glob.glob(path + '/images/*.bmp')
    
    return masks, images

batches_images = []
batches_masks = []
for i in range(8):
    path_masks, path_images = get_masks_and_images('/mnt/data/med_data/balanced_batches/batch_1/train_set/aug_set/set_1024_10k_wo_4_4/batches/batch_{i]')
    batches_images.append(path_images)
    batches_masks.append(path_masks)

In [6]:
batches_images[0]

[]

In [ ]:
import makiflow as mf
mf.set_main_gpu(0)

In [ ]:
import makiflow
from makiflow.generators.gen_base import SegmentIterator, PathGenerator
from glob import glob
import os
import numpy as np
from sklearn.utils import shuffle

In [ ]:
test_1 = None

In [ ]:
test_2 = None

In [ ]:
class SubCyclicGenerator_test(PathGenerator):
    def __init__(self, path_batches_images, path_batches_masks):
        """
        Generator for pipeline, which gives next element in sub-cyclic order
        Parameters
        ----------
        path_batches_masks : list
            A list of groups of paths to masks.
        path_batches_images : list
            A list of groups of paths to images.
        """
        assert (len(path_batches_images) == len(path_batches_masks))
        global test_1, test_2
        print('origin')
        self.batches_images = path_batches_images
        self.batches_masks = path_batches_masks
        
        self.batches_images = shuffle(self.batches_images)
        self.batches_masks = shuffle(self.batches_masks)
        
        for i in range(len(self.batches_images)):
            self.batches_images[i] = shuffle(self.batches_images[i])
            self.batches_masks[i] = shuffle(self.batches_masks[i])
        
        print('after shuffle')
        test_2 = self.batches_masks
        test_1 = self.batches_images
        
    def next_element(self):
        current_batch = 0
        counter_batches = [0 for _ in range(len(self.batches_images))]
        while True:
            if current_batch == (len(self.batches_images) - 1) and counter_batches[-1] == (
                    len(self.batches_images[-1]) - 1):
                self.batches_images, self.batches_masks = shuffle(self.batches_images, self.batches_masks)

                self.batches_images = [shuffle(elem) for elem in self.batches_images]
                self.batches_masks = [shuffle(elem) for elem in self.batches_masks]

                current_batch = 0
                counter_batches = [0 for _ in range(len(self.batches_images))]

            el = {
                SegmentIterator.image: self.batches_images[current_batch][counter_batches[current_batch]],
                SegmentIterator.mask: self.batches_masks[current_batch][counter_batches[current_batch]]
            }

            counter_batches[current_batch] += 1
            current_batch = (current_batch + 1) % len(self.batches_images)

            yield el

In [ ]:
from makiflow.models.segmentation.map_methods import LoadDataMethod, ComputePositivesPostMethod
from makiflow.models.segmentation.map_methods import NormalizePostMethod, RGB2BGRPostMethod, SqueezeMaskPostMethod

from makiflow.models.segmentation.gen_layers import InputGenLayer

def get_generator(path_images, path_masks):
    map_method = LoadDataMethod(image_shape=[1024, 1024, 3], mask_shape=[1024, 1024, 3])
    map_method = SqueezeMaskPostMethod()(map_method)
    map_method = RGB2BGRPostMethod()(map_method)
    map_method = NormalizePostMethod(use_float64=True)(map_method)
    map_method = ComputePositivesPostMethod()(map_method)
    return InputGenLayer(
        prefetch_size=8,
        batch_size=8, 
        path_generator=SubCyclicGenerator_test(path_batches_images=path_images,path_batches_masks=path_masks),
        name='Input',
        map_operation=map_method,
        num_parallel_calls=5
    )

In [ ]:
def get_masks_and_images(path):
    masks = glob(path + '/masks/*.bmp')
    images = glob(path + '/images/*.bmp')
    
    return masks, images

all_masks, all_images = get_masks_and_images('/mnt/data/med_data/balanced_batches/paper_batch_1/train_set/aug_set/set_1024_wo_4_2')

masks_1 = all_masks[:3]
images_1 = all_images[:3]

masks_2 = all_masks[500:503]
images_2 = all_masks[500:503]

masks_3 = all_masks[1000:1003]
images_3 = all_masks[1000:1003]

masks_4 = all_masks[1500:1503]
images_4 = all_masks[1500:1503]

masks_5 = all_masks[2000:2003]
images_5 = all_masks[2000:2003]

batches_images = [images_1, images_2, images_3, images_4, images_5]
batches_masks = [masks_1, masks_2, masks_3, masks_4, masks_5]

gen = get_generator(path_images=batches_images,
                    path_masks=batches_masks)

In [ ]:
test_1

In [ ]:
test_2

In [ ]:
it = gen.get_iterator()

In [ ]:
import tensorflow as tf

In [ ]:
ses = tf.Session()

In [ ]:
test = it['mask']

In [ ]:
it.

In [ ]:
for _ in range(10**9):
    ses.run(test)

In [ ]:
def get_masks_and_images(path):
    masks = glob.glob(path + '/masks/*.bmp')
    images = glob.glob(path + '/images/*.bmp')
    
    return masks, images

Ytest, Xtest = get_masks_and_images(path='/mnt/data/med_data/balanced_batches/batch_1/test_set_wo_4')

masks_1, images_1 = get_masks_and_images('/mnt/data/med_data/balanced_batches/paper_batch_1/train_set/aug_set/set_1024_wo_4_3/batch_1')
masks_2, images_2 = get_masks_and_images('/mnt/data/med_data/balanced_batches/paper_batch_1/train_set/aug_set/set_1024_wo_4_3/batch_2')
masks_3, images_3 = get_masks_and_images('/mnt/data/med_data/balanced_batches/paper_batch_1/train_set/aug_set/set_1024_wo_4_3/batch_3')
masks_4, images_4 = get_masks_and_images('/mnt/data/med_data/balanced_batches/paper_batch_1/train_set/aug_set/set_1024_wo_4_3/batch_4')

batches_images = [images_1, images_2, images_3, images_4]
batches_masks = [masks_1, masks_2, masks_3, masks_4]

In [ ]:
i = 12
two = Xtest[i]
one = Ytest[i]

In [ ]:
test_mask = cv2.imread(one)
test_imgs = cv2.imread(two)


In [ ]:
import seaborn as sb

In [ ]:
sb.heatmap(test_mask[:,:,0])

In [ ]:
np.max(test_mask[:,:,0])

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(test_imgs)

In [ ]:
import numpy as np
import copy


def mutate_masks(masks, mapping):
    """
    Remaps classes on the given `masks` according to the `mapping`.
    Parameters
    ----------
    masks : list or numpy.array
        List or numpy array of masks.
    mapping : list
        List of tuples: [(source_class_number, new_class_number)],
        where `source_class_number` will be changed to `new_class_number` in the `masks`.
    Returns
    ---------
    new_masks : the same type as `masks`
        New masks with changed class numbers.
    """
    if type(mapping) is not list or (len(mapping) != 0 and type(mapping[0]) is not tuple):
        raise TypeError('mapping should be list of typles')

    new_masks = copy.deepcopy(masks)

    for i in range(len(new_masks)):
        for elem in mapping:
            old_value = elem[0]
            new_value = elem[1]
            new_masks[i][masks[i] == old_value] = new_value

    return  new_masks

In [ ]:
def get_data(path):
    Xtrain, Ytrain = [], []
    masks = glob.glob(path + '/masks/*.bmp')
    masks.sort()
    for mask_name in tqdm(masks):
        img = cv2.imread(mask_name.replace('masks', 'images'))
        mask = cv2.imread(mask_name)
        mask = mask[:,:,0]
        Xtrain.append(img)
        Ytrain.append(mask)
        if np.max(mask) >= 10:
            print(np.max(mask), f' in image {mask_name} ')
    return Xtrain, Ytrain

In [ ]:
X,Y = get_data('/mnt/data/med_data/balanced_batches/paper_batch_3/train_set/orig_set')

In [ ]:
Y = mutate_masks(Y,mapping=[(4,0),(5,4),(6,5),(7,6),(8,7),(9,8)])

In [ ]:
def save_imgs(imgs, masks, path_to_save):
    masks_path = os.path.join(path_to_save, 'masks')
    imgs_path = os.path.join(path_to_save, 'images')
    os.makedirs(masks_path, exist_ok=True)
    os.makedirs(imgs_path, exist_ok=True)
    for i, (img, mask) in enumerate(zip(imgs, masks)):
        cv2.imwrite(masks_path+f'/{i}.bmp', mask)
        cv2.imwrite(imgs_path + f'/{i}.bmp', img)

In [ ]:
save_imgs(X, Y, path_to_save='/mnt/data/med_data/balanced_batches/paper_batch_3/train_set/orig_set_wo_4')

In [ ]:
batch = 3

In [ ]:
test = []
from glob import glob
import os
import numpy as np
from sklearn.utils import shuffle

In [ ]:
test.append(glob(os.path.join('/mnt/data/med_data/balanced_batches/batch_1/train_set/aug_set/set_1024_0/masks', '*.bmp')))

In [ ]:
test.append(glob(os.path.join('/mnt/data/med_data/balanced_batches/batch_1/train_set/aug_set/set_1024_0/images', '*.bmp')))

In [ ]:
a = glob(os.path.join('/mnt/data/med_data/balanced_batches/paper_batch_1/train_set/aug_set/set_1024_wo_4_2/images', '*.bmp'))

In [ ]:
len(a)

In [ ]:
Xtrain, Ytrain = get_data(path=f'/mnt/data/med_data/balanced_batches/batch_{batch}/train_set/aug_set/set_1024_0')
Xtest,Ytest = get_data(path=f'/mnt/data/med_data/balanced_batches/batch_{batch}/test_set')

In [ ]:
#mapping = [(0,0), (1,0), (2,0), (3,0), (4,1), (5,0), (6,0), (7,0), (8,0), (9,0)]

#mapping = [(4,0), (5,4), (6,5), (7,6), (8,7), (9,8)]

#mapping = [(0,0), (1,0), (2,0), (3,0), (4,0), (5,0), (6,0), (7,1), (8,0), (9,0)]

#mapping = [(4,0), (5,4), (6,5), (7,0), (8,6), (9,7)]

#mapping = [(0,0), (1,0), (2,0), (3,0), (4,0), (5,0), (6,0), (7,0), (8,1), (9,0)]

#mapping = [(4,0), (5,4), (6,5), (7,0), (8,0), (9,6)]

#mapping = [(0,0), (1,0), (2,0), (3,0), (4,0), (5,0), (6,1), (7,0), (8,0), (9,0)]

mapping = [(4,0), (5,4), (6,0), (7,0), (8,0), (9,5)]

In [ ]:
Ytrain = mutate_masks(Ytrain, mapping)

In [ ]:
Ytest = mutate_masks(Ytest, mapping)

In [ ]:
new_X = []
new_Y = []
for i in range(len(Ytrain)):
    if np.sum(Ytrain[i] == 1) != 0:
        new_X.append(Xtrain[i])
        new_Y.append(Ytrain[i])

Xtrain = new_X
Ytrain = new_Y

In [ ]:
new_X = []
new_Y = []
for i in range(len(Ytest)):
    if np.sum(Ytest[i] == 1) != 0:
        new_X.append(Xtest[i])
        new_Y.append(Ytest[i])
Xtest = new_X
Ytest = new_Y

In [ ]:
len(Ytest)


In [ ]:
len(Ytrain)

In [ ]:
def save_imgs(imgs, path_to_save='/mnt/data/med_data/all_data/'):
    imgs_path = os.path.join(path_to_save, 'images')
    os.makedirs(imgs_path, exist_ok=True)
    for i, img in enumerate(imgs):
        cv2.imwrite(imgs_path + f'/{i}.bmp', img)

In [ ]:
save_imgs(Xtest,path_to_save=f'/mnt/data/med_data/balanced_batches/batch_{batch}/exp_4/test_set/other_classes')
save_imgs(Xtrain,path_to_save=f'/mnt/data/med_data/balanced_batches/batch_{batch}/exp_4/train_set/other_classes')

In [ ]:
def save_masks(masks, path_to_save='/mnt/data/med_data/all_data/'):
    os.makedirs(path_to_save, exist_ok=True)
    for i, mask in enumerate(masks):
        cv2.imwrite(path_to_save+f'/{i}.bmp', mask)

In [ ]:
save_masks(Ytrain,f'/mnt/data/med_data/balanced_batches/batch_{batch}/exp_4/train_set/other_classes/masks')

In [ ]:
save_masks(Ytest,f'/mnt/data/med_data/balanced_batches/batch_{batch}/exp_4/test_set/other_classes/masks')

In [ ]:
images_path = '/mnt/data/med_data/pipeline_data_5k/images/'
masks_path = '/mnt/data/med_data/pipeline_data/masks/'

In [ ]:
images = glob.glob(os.path.join(images_path, '*.bmp'))

In [ ]:
masks = glob.glob(os.path.join(masks_path, '*.bmp'))

In [ ]:
tf.image.convert_image_dtype()

In [ ]:
masks[:10]

In [ ]:
images[:10]

In [ ]:
a_img=cv2.imread('/mnt/data/med_data/pipeline_data/images/beta__31584.bmp')
b_mask = cv2.imread('/mnt/data/med_data/pipeline_data/masks/beta__31584.bmp')
tru_img = cv2.cvtColor(a_img, cv2.COLOR_BGR2RGB).astype(np.float64)

In [ ]:
ses=tf.Session()

In [ ]:
plac = tf.placeholder(dtype=np.float32,shape=[1024,1024,3],name='int_1')

div_1 = plac / 255

In [ ]:
img_file = tf.io.read_file('/mnt/data/med_data/pipeline_data/images/beta__31584.bmp')
mask_file = tf.io.read_file('/mnt/data/med_data/pipeline_data/masks/beta__31584.bmp')
img_test = tf.io.read_file('/mnt/data/med_data/pipeline_data/images/beta__31584.bmp')

#div = tf.Variable(255.0,dtype=tf.float32)
div = tf.constant(255.0,dtype=tf.float64)
img = tf.image.decode_image(img_file)
mask_tf = tf.image.decode_image(mask_file)
img_test_tf = tf.image.decode_bmp(img_test)

img = tf.cast(img, dtype=tf.float64)
mask = tf.cast(mask_tf, dtype=tf.int32)
img_test_load = tf.image.convert_image_dtype(img_test_tf, dtype=tf.float32)

#divide = tf.divide(img,255)
divide = img / div

divide_test = tf.cast(divide, dtype=tf.float32)

In [ ]:
ses.run(tf.global_variables_initializer())

In [ ]:
img_tf = ses.run(img)
mask_tf = ses.run(mask)
nump_tf = ses.run(divide)
#nump_tf = ses.run(divide)
test = ses.run(divide_test)

In [ ]:
np.mean(ans_cv == test)

In [ ]:
ans_cv[0][0]

In [ ]:
test[0][0]

In [ ]:
import matplotlib.pyplot as plt



In [ ]:
(tru_img[0][0].astype(np.float64) / np.float64(255)).astype(np.float32)

img_tf[0][0]

In [ ]:
test[0][0]

In [ ]:
print((z*255).astype(np.int32))

In [ ]:
np.mean(test == img_tf)

In [ ]:
plt.imshow(test)
plt.show()

In [ ]:
plt.imshow(img_tf.astype(np.int32))
plt.show()


In [ ]:
ans = ses.run(divide)

In [ ]:
ans_cv = (tru_img / 255).astype(np.float32)

In [ ]:
b_mask = b_mask[:,:,0]

In [ ]:
mask_shape_np = b_mask.shape
area = mask_shape_np[0]*mask_shape_np[1]
num_neg_np = np.sum(b_mask == 0)
num_positives_np = area - num_neg

In [ ]:
ses.run(mask).shape

In [ ]:

mask = mask_tf
mask_shape = [1024,1024,1]
area = mask_shape[0] * mask_shape[1]  # tf.int32
num_neg = tf.reduce_sum(tf.cast(tf.equal(mask, 0), dtype=tf.int32))

num_positives = tf.cast(area - num_neg, dtype=tf.float32)


In [ ]:
answer_neg = ses.run(num_neg)
answer_posit = ses.run(num_positives)

In [ ]:
answer_neg

In [ ]:
num_neg_np

In [ ]:
b_mask.shape

In [ ]:
np.mean(tru_img == img_tf)

In [ ]:
np.mean(ans_cv == ans)

In [ ]:
ans[0][2]

In [ ]:
ans_cv[0][2]

In [ ]:
type(ans_cv[0][0][0])

In [ ]:
one_mask = b_mask[:,:,0]

In [ ]:
one_mask

In [ ]:
mask_tf

In [ ]:
np.mean(mask_tf == one_mask)

In [ ]:
folder = []
for i in os.walk('/home/rustam/EyePit/Result/'):
    folder.append(i)

In [ ]:
fold = []
for address, dirs, files in folder:
    for file in files:
        fold.append(address)

In [ ]:
new_fold = []
i = 0
for name in set(fold):
    if '.ipynb_checkpoints' not in name:
        new_fold.append(name)
        i += 1
fold = set(new_fold)

In [ ]:
fold.remove('/home/rustam/EyePit/Result/')

In [ ]:
len(fold)

In [ ]:
fold

In [ ]:
connect = []

In [ ]:

sez = set()

In [ ]:
new_X = []
new_Y = []

In [ ]:
m = 0
for name in fold:
    answer = None
    for i in range(1,10):
        if answer is None:
            answer = (cv2.imread(name + f'/{i*10}r.bmp')[:,:,0] // 255) * i
        else:
            read_img = cv2.imread(name + f'/{i*10}r.bmp')
            if read_img is not None:
                read_img = read_img[:,:,0]
                read_img //= 255
                read_img *= i
                if m == 4:
                    for k in connect:
                        sez.add(read_img[k[0],k[1]])
                answer += read_img
                        
    for i in range(answer.shape[0]):
        for j in range(answer.shape[1]):
            if answer[i,j] >= 10:
                answer[i,j] = 200
                #if m == 4:
                #    print(i,' ',j)
                #    connect.append((i,j))
        
        img = cv2.imread(name + '/image.bmp')
    print(name)
    if answer is not None:
        new_X.append(img)
        new_Y.append(answer)
    m += 1
    if m == 5:
        break
        

In [ ]:
sez

In [ ]:
connect

In [ ]:
import seaborn as sb
import matplotlib.pyplot as plt

In [ ]:
len(new_Y)

In [ ]:
q = 4

In [ ]:
plt.imshow(new_X[q])
plt.show()

In [ ]:
sb.heatmap(new_Y[q])

In [ ]:
sb.heatmap(new_Y[3])

In [ ]:
for name in fold:
    answer = None
    for i in range(1,10):
        if answer is None:
            answer = (cv2.imread(name + f'/{i*10}r.bmp')[:,:,0] // 255) * i
        else:
            read_img = cv2.imread(name + f'/{i*10}r.bmp')
            if read_img is not None:
                read_img = read_img[:,:,0]
                read_img //= 255
                read_img *= i
                for r in range(read_img.shape[0]):
                    for k in range(read_img.shape[1]):
                        answer[r,k] = max(answer[r,k], read_img[r,k])
                        
        img = cv2.imread(name + '/image.bmp')
    print(name)
    if answer is not None:
        new_X.append(img)
        new_Y.append(answer)
        

In [ ]:
len(new_Y)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sb

In [ ]:
i = 0

In [ ]:
sb.heatmap(new_Y[i][:,:])

In [ ]:
plt.imshow(new_X[i])
plt.show()

In [ ]:
new_X[i].shape

In [ ]:
from makiflow.augmentation import AffineAugment, ElasticAugment, ImageCutter, Data, FlipAugment, RotateAugment
import matplotlib.pyplot as plt
import seaborn as sb

In [9]:
def get_train_data():
    Xtrain, Ytrain = [], []
    masks = glob.glob('/mnt/data/med_data/balanced_batches/batch_3/train_set/aug_set/set_1024_10k_wo_4_4/origin_batch/masks/*.bmp')
    masks.sort()
    for mask_name in tqdm(masks):
        img = cv2.imread(mask_name.replace('masks', 'images'))
        mask = cv2.imread(mask_name)
        mask = mask[:,:,0]
        Xtrain.append(img)
        Ytrain.append(mask)
        if np.max(mask) >= 10:
            print(np.max(mask), f' in image {mask_name} ')
    return Xtrain, Ytrain

In [ ]:
Xtrain, Ytrain = get_train_data()

In [ ]:
def get_test_data_crop(imgs,lbls):   
    
    cropped_images, cropped_masks, _ = ImageCutter.image_and_mask_cutter(imgs, lbls,
        window_h=512, window_w=512, step_x=300, step_y=300, scale_factor=0.01)
        
    class_power = {i: 0 for i in range(9)}
    for img in cropped_masks:
        u = np.unique(img)
        for uniq in u:
            class_power[uniq] += 1
    
    print([round(i / len(cropped_images), 2) for _,i in class_power.items()] , ' power of test set')
    print(class_power)
    return cropped_images, cropped_masks

In [ ]:
Xtrain, Ytrain = get_test_data_crop(Xtrain,Ytrain)

In [ ]:
len(Xtrain)

In [13]:
all_data_images = []
all_data_masks = []
Xtrain, Ytrain = get_train_data()
all_data_images.append(Xtrain)
all_data_masks.append(Ytrain)

100%|██████████| 873/873 [00:03<00:00, 271.33it/s]


In [14]:
len(Xtrain)

873

In [15]:
#Xtrain, Ytrain = get_train_data()
alpha = 500

for i in range(11):
    Xtrain, Ytrain = all_data_images[0], all_data_masks[0]
    data = Data(Xtrain, Ytrain)
    data = ElasticAugment(alpha=alpha,num_maps=1, std=11, noise_invert_scale=7, border_mode='reflect', keep_old_data=False)(data)
    #data = RotateAugment([RotateAugment.ROTATE_90_CLOCKWISE, RotateAugment.ROTATE_90_COUNTERCLOCKWISE], keep_old_data=True)(data)
    Xtrain, Ytrain = data.get_data()
    
    all_data_images.append(Xtrain)
    all_data_masks.append(Ytrain)
    alpha += 40
    
    #Xtrain, Ytrain = data.get_data()
print(len(all_data_masks) * len(all_data_masks[0]))

10476


In [16]:
def save_imgs(imgs, masks, spec_name, path_to_save):
    masks_path = os.path.join(path_to_save, 'masks')
    imgs_path = os.path.join(path_to_save, 'images')
    os.makedirs(masks_path, exist_ok=True)
    os.makedirs(imgs_path, exist_ok=True)
    for i, (img, mask) in enumerate(zip(imgs, masks)):
        cv2.imwrite(masks_path+f'/{spec_name}_{i}.bmp', mask)
        cv2.imwrite(imgs_path + f'/{spec_name}_{i}.bmp', img)

In [ ]:
save_imgs(Xtrain,Ytrain,'alpha', path_to_save='/mnt/data/med_data/balanced_batches/batch_3/train_set/aug_set/set_1024_10k_wo_4_4/10k')

In [17]:
for i in range(len(all_data_masks)):
    save_imgs(all_data_images[i],all_data_masks[i],f'alpha_num_batch_{i}', 
              path_to_save=f'/mnt/data/med_data/balanced_batches/batch_3/train_set/aug_set/set_1024_10k_wo_4_4/batches/batch_{i}')

In [ ]:
i = 0
masks = glob.glob('/mnt/data/med_data/pipeline_data_8k/masks/*.bmp')
for s in masks:
    i += 1
print(i)

In [ ]:
import seaborn as sb
import matplotlib.pyplot as plt

In [ ]:
sb.heatmap(all_data_masks[1][234])

In [ ]:
plt.imshow(all_data_images[1][234])

In [ ]:
from generator_provider import Generator

In [ ]:
images_path = '/mnt/data/med_data/pipeline_data_8k/images/'
masks_path = '/mnt/data/med_data/pipeline_data_8k/masks/'

In [ ]:
gen = Generator(images_path, masks_path)

In [ ]:
it = gen.next_element()

In [ ]:
it

In [ ]:
for i, _ in enumerate(it):
    if i == 0:
        continue
    if i % 1 == 0:
        print(i)
    if i % 100000 == 0:
        break

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
size = 8000
it = size*10

In [ ]:
image = dict([(i,0) for i in range(size)])

In [ ]:
image

In [ ]:
for _ in range(it):
    index = np.random.randint(low=0,high=len(image))
    image[index] += 1

In [ ]:
image

In [ ]:
y = np.array([i for k,i in image.items()])

In [ ]:
x = np.array([i for i in range(size)])

In [ ]:
y = np.sort(y)

In [ ]:
plt.bar(x, y, align='edge')
plt.show()

In [ ]:
image = dict([(i,0) for i in range(size)])

In [ ]:

index = 0 
for _ in range(it):
    if index % len(image) == 0 and index != 0:
        index = 0
    image[index] += 1
    index += 1

In [ ]:
image

In [ ]:
y_1 = np.array([i for k,i in image.items()])

In [ ]:
x_1 = np.array([i for i in range(size)])

In [ ]:
plt.bar(x_1, y_1, align='edge')
plt.show()

In [ ]:
from makiflow.models.segmentation import Segmentator

In [ ]:
from makiflow.layers import *


In [ ]:
in_x = InputLayer(input_shape=[1,10,10,3],name='inp')
x = ConvLayer(kh=3,kw=3,in_f=1,out_f=10,name='conv1')(in_x)
model

In [ ]:
def count(lbls, classes):
    uniq = dict([(str(i),0) for i in range(classes)])
    
    for i in range(len(lbls)):
        a,_ = np.unique(lbls[i], return_counts=True)
        for num in a:
            uniq[str(num)] += 1
    return [round(uniq[k] / len(lbls),2) for k in uniq]

In [ ]:
answer = count(Ytrain, 10)

In [ ]:
answer

In [ ]:
[i * len(Ytrain) for i in answer]

In [ ]:
Xtrain = [cv2.imread('/mnt/data/med_data/balanced_batches/batch_1/test_set/images/1.bmp')]
Ytrain = [cv2.imread('/mnt/data/med_data/balanced_batches/batch_1/test_set/masks/1.bmp')]

In [ ]:
data = Data(Xtrain, Ytrain)
data = ElasticAugment(alpha=500,num_maps=1, std=14, noise_invert_scale=7, border_mode='reflect_101', keep_old_data=False)(data)

Xtrain, Ytrain = data.get_data()
print(len(Xtrain))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
plt.imshow(Xtrain[0])
plt.show()

In [ ]:
sb.heatmap(Ytrain[0][:,:,0])

In [ ]:
del Xtrain,Ytrain